# Thermal IR Tutorial 2: Comparing raster and point datasets

**At the conclusion of this tutorial, you will be able to:**
- Visualize point and raster thermal infrared datasets together
- Compute error statistics between point and raster datasets